## Script for Plotting & Stats

In [ ]:
%matplotlib inline
import os
import mne
import pandas as pd
import numpy as np
from neurora.stuff import permutation_corr
from mne.viz import plot_topomap
from mne.stats import fdr_correction
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from jupyterthemes import jtplot
jtplot.style(theme='grade3') 
from Function import spearmanr, permutation_cor

# Define the path
eeg_path = 'F:/1_Emotion_Data/Results/0_ISC/EEG/'
behav_path = 'F:/1_Emotion_Data/Data/Behavior/Data/Social_Scale/'
physio_path = 'F:/1_Emotion_Data/Results/0_ISC/ISC_CSV/Physiology/'
results_path = 'F:/1_Emotion_Data/Results/1_IS_RSA/'
plot_path = 'F:/1_Emotion_Data/Results/2_Plots/'

In [ ]:
# Use the mne.read_epochs_eeglab() to read the preprocessed data
demo = mne.read_epochs_eeglab('F:/1_Emotion_Data/Data/EEG/Angry/ag1/sub_001_ag1.set')
montage = mne.channels.read_custom_montage('F:/1_Emotion_Data/Data/EEG/Emotion.loc')
demo.set_montage(montage)

# Obtain channel & montage information for topo-plot
topo_info = demo.info
# Obtain the channel names as a list
ch_names = demo.ch_names
# Crate a index list for channels
ch_idx = list(range(63))
# Combine the channels and index and convert to a dict
ch = dict(zip(ch_names, ch_idx))
print(ch['TP8'])

In [ ]:
def isc_topo(r_topo,topo_info,vmin,vmax,title):
    fig,(ax1, ax2, ax3, ax4, ax5) = plt.subplots(ncols=5, figsize=(25,15))
    im,_ = plot_topomap(r_topo[:,0], topo_info, axes=ax1, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)
    im,_ = plot_topomap(r_topo[:,1], topo_info, axes=ax2, show=False, vmin=0, vmax=0.15,  cmap='Reds',sphere=0.13)    
    im,_ = plot_topomap(r_topo[:,2], topo_info, axes=ax3, show=False, vmin=0, vmax=0.15,  cmap='Reds',sphere=0.13)    
    im,_ = plot_topomap(r_topo[:,3], topo_info, axes=ax4, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)    
    im,_ = plot_topomap(r_topo[:,4], topo_info, axes=ax5, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)  
    # ax1.set_title('Delta',fontsize=20) 
    # ax2.set_title('Theta',fontsize=20) 
    # ax3.set_title('Alpha',fontsize=20) 
    # ax4.set_title('Beta',fontsize=20) 
    # ax5.set_title('Gamma',fontsize=20) 
    ax_x_start, ax_x_width, ax_y_start, ax_y_height  = 0.95, 0.015, 0.4, 0.25
    cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
    clb = fig.colorbar(im, cax=cbar_ax)
    #fig.suptitle(title, x=0.5, y=0.75, fontsize=30, fontweight='bold')
    plt.show()
    return fig

In [ ]:
def mean_isc(r_topo):
    print('The overall isc value of delta is ' + str(r_topo[:,0].mean()))
    print('The overall isc value of theta is ' + str(r_topo[:,1].mean()))
    print('The overall isc value of alpha is ' + str(r_topo[:,2].mean()))
    print('The overall isc value of beta is ' + str(r_topo[:,3].mean()))
    print('The overall isc value of gamma is ' + str(r_topo[:,4].mean()))

In [ ]:
def isrsa_topo(isrsa_matrix,topo_info,vmin,vmax,title):
    fig,(ax1, ax2, ax3, ax4, ax5) = plt.subplots(ncols=5, figsize=(25,15))

    mask = obtain_mask(isrsa_matrix['delta_p_cor'], ch_idx)
    im,_ = plot_topomap(isrsa_matrix['delta_r'], topo_info, axes=ax1, show=False, mask=mask, mask_params=dict(marker='o',markersize=7, markerfacecolor='black'), vmin=vmin, vmax=vmax,cmap='RdBu_r',sphere=0.13)
    mask = obtain_mask(isrsa_matrix['theta_p_cor'], ch_idx)
    im,_ = plot_topomap(isrsa_matrix['theta_r'], topo_info, axes=ax2, show=False, mask=mask, mask_params=dict(marker='o',markersize=7, markerfacecolor='black'), vmin=vmin, vmax=vmax,  cmap='RdBu_r',sphere=0.13)
    mask = obtain_mask(isrsa_matrix['alpha_p_cor'], ch_idx)    
    im,_ = plot_topomap(isrsa_matrix['alpha_r'], topo_info, axes=ax3, show=False, mask=mask, mask_params=dict(marker='o',markersize=7, markerfacecolor='black'), vmin=vmin, vmax=vmax,  cmap='RdBu_r',sphere=0.13)
    mask = obtain_mask(isrsa_matrix['beta_p_cor'], ch_idx)    
    im,_ = plot_topomap(isrsa_matrix['beta_r'], topo_info, axes=ax4, show=False, mask=mask, mask_params=dict(marker='o',markersize=7, markerfacecolor='black'), vmin=vmin, vmax=vmax, cmap='RdBu_r',sphere=0.13)
    mask = obtain_mask(isrsa_matrix['gamma_p_cor'], ch_idx)    
    im,_ = plot_topomap(isrsa_matrix['gamma_r'], topo_info, axes=ax5, show=False, mask=mask, mask_params=dict(marker='o',markersize=7, markerfacecolor='black'), vmin=vmin, vmax=vmax, cmap='RdBu_r',sphere=0.13)

    ax1.set_title('Delta',fontsize=20) 
    ax2.set_title('Theta',fontsize=20) 
    ax3.set_title('Alpha',fontsize=20) 
    ax4.set_title('Beta',fontsize=20) 
    ax5.set_title('Gamma',fontsize=20) 
    ax_x_start, ax_x_width, ax_y_start, ax_y_height  = 0.95, 0.02, 0.4, 0.3
    cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
    clb = fig.colorbar(im, cax=cbar_ax)
    fig.suptitle(title, x=0.5, y=0.75, fontsize=25, fontweight='bold')
    plt.show()
    # plt.savefig('F:/1_Emotion_Data/Results/2_Plots/ISRSA/ag_so.png',bbox_inches='tight',dpi=600,pad_inches=0.1)
    # plt.close()
    return fig

In [ ]:
# Obtain mask
def obtain_mask(p_vector, ch_idx):  
    tp = np.array(p_vector)
    for i in ch_idx: 
        p = tp[i]
        if p <= 0.05:
            tp[i] = True
        else:
            tp[i] = False
    return tp

### Section 1: ISC Topoplot

#### Angry

In [ ]:
ag_isc = np.load(os.path.join(eeg_path, 'ag_isc_matrix.npy'), allow_pickle=True).item()
ag_r_topo = ag_isc['r_topo']
topo = isc_topo(ag_r_topo,topo_info,vmin=0,vmax=0.15,title='Angry PSD Intersubject Similarity')
#topo.savefig(os.path.join(plot_path,'ISC/EEG/ag_isc.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
mean_isc(ag_r_topo)

### Anxiety

In [ ]:
ax_isc = np.load(os.path.join(eeg_path, 'ax_isc_matrix.npy'), allow_pickle=True).item()
ax_r_topo = ax_isc['r_topo']
topo = isc_topo(ax_r_topo,topo_info,vmin=0,vmax=0.15,title='Anxiety PSD Intersubject Similarity')
#topo.savefig(os.path.join(plot_path,'ISC/EEG/ax_isc.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
mean_isc(ax_r_topo)

### Fear

In [ ]:
fe_isc = np.load(os.path.join(eeg_path, 'fe_isc_matrix.npy'), allow_pickle=True).item()
fe_r_topo = fe_isc['r_topo']
topo = isc_topo(fe_r_topo,topo_info,vmin=0,vmax=0.15,title='Fear PSD Intersubject Similarity')
#topo.savefig(os.path.join(plot_path,'ISC/EEG/fe_isc.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
mean_isc(fe_r_topo)

### Helpless

In [ ]:
hl_isc = np.load(os.path.join(eeg_path, 'hl_isc_matrix.npy'), allow_pickle=True).item()
hl_r_topo = hl_isc['r_topo']
topo = isc_topo(hl_r_topo,topo_info,vmin=0,vmax=0.15,title='Helpless PSD Intersubject Similarity')
#topo.savefig(os.path.join(plot_path,'ISC/EEG/hl_ss.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
mean_isc(hl_r_topo)

### Happy

In [ ]:
ha_isc = np.load(os.path.join(eeg_path, 'ha_isc_matrix.npy'), allow_pickle=True).item()
ha_r_topo = ha_isc['r_topo']
topo = isc_topo(ha_r_topo,topo_info,vmin=0,vmax=0.15,title='Happy PSD Intersubject Similarity')
#topo.savefig(os.path.join(plot_path,'ISC/EEG/ha_ss.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
mean_isc(ha_r_topo)

### Section 2: Behav 3D plot

In [ ]:
imq = pd.read_csv(os.path.join(behav_path, 'IMQ.csv'))
iri = pd.read_csv(os.path.join(behav_path, 'IRI.csv'))
behav_summary = pd.concat([imq['SO'], imq['SS'], imq['OS'], iri['EC'], iri['PD'], iri['FS'], iri['PT']], axis=1)

In [ ]:
fig = px.scatter_3d(behav_summary, x='SO', y='SS', z='EC')
fig.show()

### Section 3: IS-RSA Topoplot

### Angry

In [ ]:
ag_ss = pd.read_csv(os.path.join(results_path, '1_side/ag_ss_isrsa_cor.csv'))
topo = isrsa_topo(ag_ss,topo_info,vmin=-0.15,vmax=0.15,title='Angry & IMQ-SS ISRSA')
topo.savefig(os.path.join(plot_path,'ag_ss.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
ag_so = pd.read_csv(os.path.join(results_path, '1_side/ag_so_isrsa_cor.csv'))
topo = isrsa_topo(ag_so,topo_info,vmin=-0.15,vmax=0.15,title='Angry & IMQ-SO ISRSA')
topo.savefig(os.path.join(plot_path,'ag_so.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
ag_ec = pd.read_csv(os.path.join(results_path, '1_side/ag_ec_isrsa_cor.csv'))
topo = isrsa_topo(ag_ec,topo_info,vmin=-0.15,vmax=0.15,title='Angry & IRI-EC ISRSA')
topo.savefig(os.path.join(plot_path,'ag_ec.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
ag_pd = pd.read_csv(os.path.join(results_path, '1_side/ag_pd_isrsa_cor.csv'))
topo = isrsa_topo(ag_pd,topo_info,vmin=-0.15,vmax=0.15,title='Angry & IRI-PD ISRSA')
topo.savefig(os.path.join(plot_path,'ag_pd.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

### Anxiety

In [ ]:
ax_ss = pd.read_csv(os.path.join(results_path, '1_side/ax_ss_isrsa_cor.csv'))
topo = isrsa_topo(ax_ss,topo_info,vmin=-0.15,vmax=0.15,title='Anxious & IMQ-SS ISRSA')
topo.savefig(os.path.join(plot_path,'ax_ss.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
ax_so = pd.read_csv(os.path.join(results_path, '1_side/ax_so_isrsa_cor.csv'))
topo = isrsa_topo(ax_so,topo_info,vmin=-0.15,vmax=0.15,title='Anxious & IMQ-SO ISRSA')
topo.savefig(os.path.join(plot_path,'ax_so.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
ax_ec = pd.read_csv(os.path.join(results_path, '1_side/ax_ec_isrsa_cor.csv'))
topo = isrsa_topo(ax_ec,topo_info,vmin=-0.15,vmax=0.15,title='Anxious & IRI-EC ISRSA')
topo.savefig(os.path.join(plot_path,'ax_ec.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
ax_pd = pd.read_csv(os.path.join(results_path, '1_side/ax_pd_isrsa_cor.csv'))
topo = isrsa_topo(ax_pd,topo_info,vmin=-0.15,vmax=0.15,title='Anxious & IRI-PD ISRSA')
topo.savefig(os.path.join(plot_path,'ax_pd.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

### Fear

In [ ]:
fe_ss = pd.read_csv(os.path.join(results_path, '1_side/fe_ss_isrsa_cor.csv'))
topo = isrsa_topo(fe_ss,topo_info,vmin=-0.15,vmax=0.15,title='Fearful & IMQ-SS ISRSA')
topo.savefig(os.path.join(plot_path,'fe_ss.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
fe_so = pd.read_csv(os.path.join(results_path, '1_side/fe_so_isrsa_cor.csv'))
topo = isrsa_topo(fe_so,topo_info,vmin=-0.15,vmax=0.15,title='Fearful & IMQ-SO ISRSA')
topo.savefig(os.path.join(plot_path,'fe_so.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
fe_ec = pd.read_csv(os.path.join(results_path, '1_side/fe_ec_isrsa_cor.csv'))
topo = isrsa_topo(fe_ec,topo_info,vmin=-0.15,vmax=0.15,title='Fearful & IRI-EC ISRSA')
topo.savefig(os.path.join(plot_path,'fe_ec.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
fe_pd = pd.read_csv(os.path.join(results_path, '1_side/fe_pd_isrsa_cor.csv'))
topo = isrsa_topo(fe_pd,topo_info,vmin=-0.15,vmax=0.15,title='Fearful & IRI-PD ISRSA')
topo.savefig(os.path.join(plot_path,'fe_pd.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

### Helpless

In [ ]:
hl_ss = pd.read_csv(os.path.join(results_path, '1_side/hl_ss_isrsa_cor.csv'))
topo = isrsa_topo(hl_ss,topo_info,vmin=-0.15,vmax=0.15,title='Helpless & IMQ-SS ISRSA')
topo.savefig(os.path.join(plot_path,'hl_ss.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
hl_so = pd.read_csv(os.path.join(results_path, '1_side/hl_so_isrsa_cor.csv'))
topo = isrsa_topo(hl_so,topo_info,vmin=-0.15,vmax=0.15,title='Helpless & IMQ-SO ISRSA')
topo.savefig(os.path.join(plot_path,'hl_so.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
hl_ec = pd.read_csv(os.path.join(results_path, '1_side/hl_ec_isrsa_cor.csv'))
topo = isrsa_topo(hl_ec,topo_info,vmin=-0.15,vmax=0.15,title='Helpless & IRI-EC ISRSA')
topo.savefig(os.path.join(plot_path,'hl_ec.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
hl_pd = pd.read_csv(os.path.join(results_path, '1_side/hl_pd_isrsa_cor.csv'))
topo = isrsa_topo(hl_pd,topo_info,vmin=-0.15,vmax=0.15,title='Helpless & IRI-PD ISRSA')
topo.savefig(os.path.join(plot_path,'hl_pd.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

### Happy

In [ ]:
ha_ss = pd.read_csv(os.path.join(results_path, '1_side/ha_ss_isrsa_cor.csv'))
topo = isrsa_topo(ha_ss,topo_info,vmin=-0.15,vmax=0.15,title='Happy & IMQ-SS ISRSA')
topo.savefig(os.path.join(figure_path,'ha_ss.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
ha_so = pd.read_csv(os.path.join(results_path, '1_side/ha_so_isrsa_cor.csv'))
topo = isrsa_topo(ha_so,topo_info,vmin=-0.15,vmax=0.15,title='Happy & IMQ-SO ISRSA')
topo.savefig(os.path.join(figure_path,'ha_so.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

In [ ]:
ha_fs = pd.read_csv(os.path.join(results_path, '1_side/ha_fs_isrsa_cor.csv'))
topo = isrsa_topo(ha_fs,topo_info,vmin=-0.15,vmax=0.15,title='Happy & IRI-FS ISRSA')
topo.savefig(os.path.join(figure_path,'ha_fs.png'),bbox_inches='tight',dpi=600,pad_inches=0.1)

### Physiology avearge ISC

In [ ]:
ag_physio = pd.read_csv(os.path.join(physio_path, 'ag_physio_isc.csv'))
ax_physio = pd.read_csv(os.path.join(physio_path, 'ax_physio_isc.csv'))
fe_physio = pd.read_csv(os.path.join(physio_path, 'fe_physio_isc.csv'))
hl_physio = pd.read_csv(os.path.join(physio_path, 'hl_physio_isc.csv'))
ha_physio = pd.read_csv(os.path.join(physio_path, 'ha_physio_isc.csv'))

In [ ]:
print('Angry HR mean ISC = ' + str(ag_physio['HR'].mean()))
print('Angry EDR mean ISC = ' + str(ag_physio['EDR'].mean()))
print('Angry HRV mean ISC = ' + str(ag_physio['HRV_MeanNN'].mean()))

In [ ]:
print('Anxiety HR mean ISC = ' + str(ax_physio['HR'].mean()))
print('Anxiety EDR mean ISC = ' + str(ax_physio['EDR'].mean()))
print('Anxiety HRV mean ISC = ' + str(ax_physio['HRV_MeanNN'].mean()))

In [ ]:
print('Fear HR mean ISC = ' + str(fe_physio['HR'].mean()))
print('Fear EDR mean ISC = ' + str(fe_physio['EDR'].mean()))
print('Fear HRV mean ISC = ' + str(fe_physio['HRV_MeanNN'].mean()))

In [ ]:
print('Helpless HR mean ISC = ' + str(hl_physio['HR'].mean()))
print('Helpless EDR mean ISC = ' + str(hl_physio['EDR'].mean()))
print('Helpless HRV mean ISC = ' + str(hl_physio['HRV_MeanNN'].mean()))

In [ ]:
print('Happy HR mean ISC = ' + str(ha_physio['HR'].mean()))
print('Happy EDR mean ISC = ' + str(ha_physio['EDR'].mean()))
print('Happy HRV mean ISC = ' + str(ha_physio['HRV_MeanNN'].mean()))

### Physiology ISC matrices

In [ ]:
ecg_dir = 'F:/1_Emotion_Data/Results/0_ISC/Physiology/'
physio_plot_path = 'F:/1_Emotion_Data/Results/2_Plots/ISC/Physiology/'

ag_hr_simi = np.load(os.path.join(ecg_dir, 'ag_hr_isc.npy'), allow_pickle=True)
ag_hrv_simi = np.load(os.path.join(ecg_dir, 'ag_hrvm_isc.npy'), allow_pickle=True)

ax_hr_simi = np.load(os.path.join(ecg_dir, 'ax_hr_isc.npy'), allow_pickle=True)
ax_hrv_simi = np.load(os.path.join(ecg_dir, 'ax_hrvm_isc.npy'), allow_pickle=True)

fe_hr_simi = np.load(os.path.join(ecg_dir, 'fe_hr_isc.npy'), allow_pickle=True)
fe_hrv_simi = np.load(os.path.join(ecg_dir, 'fe_hrvm_isc.npy'), allow_pickle=True)

hl_hr_simi = np.load(os.path.join(ecg_dir, 'hl_hr_isc.npy'), allow_pickle=True)
hl_hrv_simi = np.load(os.path.join(ecg_dir, 'hl_hrvm_isc.npy'), allow_pickle=True)

ha_hr_simi = np.load(os.path.join(ecg_dir, 'ha_hr_isc.npy'), allow_pickle=True)
ha_hrv_simi = np.load(os.path.join(ecg_dir, 'ha_hrvm_isc.npy'), allow_pickle=True)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
sns.heatmap(ag_hr_simi, square=True, cmap='RdBu_r', linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Angry HR ISC", fontsize=25, fontweight='bold')
sns.heatmap(ag_hrv_simi, square=True, cmap='RdBu_r', linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Angry HRV ISC", fontsize=25, fontweight='bold')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
sns.heatmap(ax_hr_simi, square=True, cmap='RdBu_r', linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Anxious HR ISC", fontsize=25, fontweight='bold')
sns.heatmap(ax_hrv_simi, square=True, cmap='RdBu_r', linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Anxious HRV ISC", fontsize=25, fontweight='bold')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
sns.heatmap(fe_hr_simi, square=True, cmap='RdBu_r', linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Fearful HR ISC", fontsize=25, fontweight='bold')
sns.heatmap(fe_hrv_simi, square=True, cmap='RdBu_r', linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Faerful HRV ISC", fontsize=25, fontweight='bold')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
sns.heatmap(hl_hr_simi, square=True, cmap='RdBu_r', linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Helpless HR ISC", fontsize=25, fontweight='bold')
sns.heatmap(hl_hrv_simi, square=True, cmap='RdBu_r', linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Helpless HRV ISC", fontsize=25, fontweight='bold')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
sns.heatmap(ha_hr_simi, square=True, cmap='RdBu_r', linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Happy HR ISC", fontsize=25, fontweight='bold')
sns.heatmap(ha_hrv_simi, square=True, cmap='RdBu_r', linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Happy HRV ISC", fontsize=25, fontweight='bold')